# FluiditySaramitoLocal: Creep with Elastic Jump and Viscous Bifurcation

This notebook demonstrates creep behavior in the FluiditySaramitoLocal model, showcasing:
- Elastic jump at t=0: γ_e(0) = σ₀/G
- Viscous bifurcation: below vs above yield stress
- Maxwell backbone contribution to transient response
- NLSQ and Bayesian parameter inference from creep data

## 1. Setup and Imports

In [ ]:
# Google Colab compatibility
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax
except ImportError:
    IN_COLAB = False

import sys
from pathlib import Path

if not IN_COLAB:
    # Add parent directory to path for local development
    notebook_dir = Path.cwd()
    project_root = notebook_dir.parent.parent
    if project_root not in [Path(p) for p in sys.path]:
        sys.path.insert(0, str(project_root))

In [ ]:
# JAX float64 configuration (CRITICAL: must be first)
from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# RheoJAX imports
from rheojax.models.fluidity import FluiditySaramitoLocal
from rheojax.core.data import RheoData
from rheojax.logging import configure_logging, get_logger

# Configure logging
configure_logging(level="INFO")
logger = get_logger(__name__)

# Set random seeds for reproducibility
np.random.seed(42)
key = jax.random.PRNGKey(42)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

## 2. Theory: Saramito Creep Response

### Governing Equations

For constant applied stress σ₀, the FluiditySaramitoLocal model exhibits:

**Total strain decomposition:**
$$\gamma(t) = \gamma_e(t) + \gamma_v(t)$$

**Elastic jump (Maxwell backbone):**
$$\gamma_e(0) = \frac{\sigma_0}{G}$$

**Viscous flow (von Mises + fluidity):**
$$\dot{\gamma}_v = \frac{1}{\eta_f} \max\left(0, 1 - \frac{\tau_y}{|\sigma_0|}\right) \sigma_0$$

where $\eta_f = \eta_0/f$ is the fluidity-dependent viscosity.

**Fluidity evolution:**
$$\frac{df}{dt} = -\frac{f - 1}{\tau_{th}} + b |\dot{\gamma}_v|^n$$

### Bifurcation Behavior

1. **Below yield (σ₀ < τ_y):**
   - Elastic jump only: γ(t) ≈ σ₀/G
   - No viscous flow: α = 0
   - Fluidity decays to 1 (aging)

2. **Above yield (σ₀ > τ_y):**
   - Elastic jump + viscous flow
   - Delayed yielding: fluidity increases, viscosity decreases
   - Terminal flow: γ(t) ~ σ₀t/(η_∞)

3. **Near yield (σ₀ ≈ τ_y):**
   - Competition between aging and rejuvenation
   - Possible transient yielding followed by arrest

## 3. Model Setup and Parameters

In [ ]:
# Create output directory
if IN_COLAB:
    output_dir = Path('/content/outputs/fluidity/saramito_local/creep')
else:
    output_dir = Path('../outputs/fluidity/saramito_local/creep')
output_dir.mkdir(parents=True, exist_ok=True)

logger.info(f"Output directory: {output_dir}")

In [ ]:
# Try to load calibrated parameters from startup tutorial
startup_params_file = output_dir.parent / 'startup' / 'calibrated_params.txt'

if startup_params_file.exists():
    logger.info(f"Loading calibrated parameters from {startup_params_file}")
    # Parse parameters (simple key=value format)
    params = {}
    with open(startup_params_file, 'r') as f:
        for line in f:
            if '=' in line and not line.strip().startswith('#'):
                key, value = line.strip().split('=')
                params[key.strip()] = float(value.strip())
    
    # Create model with calibrated parameters
    model = FluiditySaramitoLocal(coupling="minimal")
    model.parameters.set_values(
        G=params.get('G', 1000.0),
        eta_0=params.get('eta_0', 500.0),
        tau_y=params.get('tau_y', 50.0),
        tau_th=params.get('tau_th', 10.0),
        b=params.get('b', 0.1),
        n=params.get('n', 1.0)
    )
    logger.info("Using calibrated parameters from startup")
else:
    logger.info("Using default parameters (startup calibration not found)")
    # Default parameters for demonstration
    model = FluiditySaramitoLocal(coupling="minimal")
    model.parameters.set_values(
        G=1000.0,      # Pa (elastic modulus)
        eta_0=500.0,   # Pa·s (reference viscosity)
        tau_y=50.0,    # Pa (yield stress)
        tau_th=10.0,   # s (thixotropic timescale)
        b=0.1,         # s^-1 (rejuvenation rate)
        n=1.0          # (-) (shear-rate exponent)
    )

print("\nModel Parameters:")
print(model.parameters)

## 4. Generate Synthetic Creep Data

In [ ]:
# Define stress levels relative to yield stress
tau_y = model.parameters.get_value('tau_y')
G = model.parameters.get_value('G')

sigma_levels = {
    'below_yield': 0.7 * tau_y,   # 35 Pa (no flow)
    'at_yield': 1.0 * tau_y,      # 50 Pa (critical)
    'above_yield': 1.5 * tau_y    # 75 Pa (flow)
}

# Time array (longer to capture steady-state)
t_end = 200.0  # s
n_points = 500
t = np.linspace(0, t_end, n_points)

# Initial fluidity (equilibrium state)
f_0 = 1.0

# Generate creep responses
creep_data = {}
noise_level = 0.02  # 2% noise

for label, sigma in sigma_levels.items():
    logger.info(f"Simulating creep at σ = {sigma:.1f} Pa ({label})")
    
    gamma, f = model.simulate_creep(t, sigma, f_0=f_0)
    
    # Add realistic noise
    gamma_noisy = gamma * (1 + noise_level * np.random.randn(len(gamma)))
    
    # Compute elastic and viscous contributions
    gamma_elastic = sigma / G
    gamma_viscous = gamma - gamma_elastic
    
    creep_data[label] = {
        't': t,
        'gamma': gamma,
        'gamma_noisy': gamma_noisy,
        'gamma_elastic': gamma_elastic,
        'gamma_viscous': gamma_viscous,
        'fluidity': f,
        'sigma': sigma
    }

logger.info("Synthetic creep data generated")

In [ ]:
# Visualize all creep responses
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

colors = {'below_yield': 'blue', 'at_yield': 'orange', 'above_yield': 'red'}
labels_pretty = {
    'below_yield': f'σ = {sigma_levels["below_yield"]:.1f} Pa (0.7τ_y)',
    'at_yield': f'σ = {sigma_levels["at_yield"]:.1f} Pa (1.0τ_y)',
    'above_yield': f'σ = {sigma_levels["above_yield"]:.1f} Pa (1.5τ_y)'
}

# Panel 1: Total strain
ax = axes[0, 0]
for label, data in creep_data.items():
    ax.plot(data['t'], data['gamma'], '-', color=colors[label], 
            label=labels_pretty[label], linewidth=2)
    ax.plot(data['t'], data['gamma_noisy'], '.', color=colors[label], 
            alpha=0.3, markersize=3)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Strain γ (-)')
ax.set_title('Total Creep Strain')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Elastic vs viscous (above yield only)
ax = axes[0, 1]
data = creep_data['above_yield']
ax.plot(data['t'], data['gamma'], 'k-', label='Total', linewidth=2)
ax.axhline(data['gamma_elastic'], color='green', linestyle='--', 
           label=f'Elastic (σ/G = {data["gamma_elastic"]:.3f})', linewidth=2)
ax.plot(data['t'], data['gamma_viscous'], 'purple', linestyle=':', 
        label='Viscous', linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Strain γ (-)')
ax.set_title('Elastic Jump + Viscous Flow (σ > τ_y)')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 3: Fluidity evolution
ax = axes[1, 0]
for label, data in creep_data.items():
    ax.plot(data['t'], data['fluidity'], '-', color=colors[label], 
            label=labels_pretty[label], linewidth=2)
ax.axhline(1.0, color='gray', linestyle='--', alpha=0.5, label='Equilibrium (f=1)')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Fluidity f (-)')
ax.set_title('Fluidity Evolution')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 4: Log-log strain (identify power-law regime)
ax = axes[1, 1]
for label, data in creep_data.items():
    # Avoid log(0) by starting from t > 0
    mask = data['t'] > 1.0
    ax.loglog(data['t'][mask], data['gamma'][mask], '-', color=colors[label], 
              label=labels_pretty[label], linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Strain γ (-)')
ax.set_title('Log-Log Creep (identifies power-law regimes)')
ax.legend()
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig(output_dir / 'synthetic_creep_overview.png', dpi=300, bbox_inches='tight')
plt.show()

logger.info(f"Saved overview plot to {output_dir / 'synthetic_creep_overview.png'}")

## 5. NLSQ Parameter Fitting

Fit the model to the above-yield creep data to recover parameters.

In [ ]:
# Select above-yield data for fitting (most informative)
fit_data = creep_data['above_yield']
sigma_fit = fit_data['sigma']

# Create RheoData object
rheo_data = RheoData(
    x=fit_data['t'],
    y=fit_data['gamma_noisy'],
    test_mode='creep'
)

logger.info(f"Fitting to creep data at σ = {sigma_fit:.1f} Pa")
print(f"\nData points: {len(rheo_data.x)}")
print(f"Strain range: {rheo_data.y.min():.4f} to {rheo_data.y.max():.4f}")

In [ ]:
# Create fresh model for fitting
model_fit = FluiditySaramitoLocal(coupling="minimal")

# Set reasonable parameter bounds
model_fit.parameters.set_bounds('G', 100.0, 5000.0)
model_fit.parameters.set_bounds('eta_0', 100.0, 2000.0)
model_fit.parameters.set_bounds('tau_y', 10.0, 100.0)
model_fit.parameters.set_bounds('tau_th', 1.0, 50.0)
model_fit.parameters.set_bounds('b', 0.01, 1.0)
model_fit.parameters.set_bounds('n', 0.5, 2.0)

# Initial guess (perturbed from true values)
model_fit.parameters.set_values(
    G=800.0,
    eta_0=400.0,
    tau_y=40.0,
    tau_th=8.0,
    b=0.08,
    n=1.2
)

print("\nInitial guess:")
print(model_fit.parameters)

In [ ]:
# Perform NLSQ fit
logger.info("Starting NLSQ optimization...")

result = model_fit.fit(
    rheo_data,
    test_mode='creep',
    sigma_applied=sigma_fit,
    max_iter=5000,
    ftol=1e-8,
    xtol=1e-8
)

print("\n" + "="*60)
print("NLSQ Fit Results")
print("="*60)
print(f"Converged: {result.success}")
print(f"Iterations: {result.nit}")
print(f"Final cost: {result.cost:.6e}")
print(f"R²: {result.r_squared:.6f}")
print(f"RMSE: {result.rmse:.6e}")
print("\nFitted parameters:")
print(model_fit.parameters)

# Compare with true values
print("\nTrue vs Fitted:")
for param_name in ['G', 'eta_0', 'tau_y', 'tau_th', 'b', 'n']:
    true_val = model.parameters.get_value(param_name)
    fit_val = model_fit.parameters.get_value(param_name)
    error_pct = 100 * abs(fit_val - true_val) / true_val
    print(f"{param_name:8s}: True={true_val:8.3f}, Fit={fit_val:8.3f}, Error={error_pct:5.1f}%")

In [ ]:
# Visualize NLSQ fit
gamma_pred = model_fit.predict(rheo_data.x, test_mode='creep', sigma_applied=sigma_fit)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Fit quality
ax = axes[0]
ax.plot(fit_data['t'], fit_data['gamma'], 'k-', label='True', linewidth=2, alpha=0.7)
ax.plot(fit_data['t'], fit_data['gamma_noisy'], 'o', color='gray', 
        label='Data (noisy)', markersize=3, alpha=0.5)
ax.plot(rheo_data.x, gamma_pred, 'r--', label='NLSQ Fit', linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Strain γ (-)')
ax.set_title(f'NLSQ Fit (R² = {result.r_squared:.4f})')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Residuals
ax = axes[1]
residuals = rheo_data.y - gamma_pred
ax.plot(rheo_data.x, residuals, 'o', color='red', markersize=3, alpha=0.6)
ax.axhline(0, color='black', linestyle='--', linewidth=1)
ax.fill_between(rheo_data.x, -2*result.rmse, 2*result.rmse, 
                alpha=0.2, color='red', label='±2 RMSE')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Residuals')
ax.set_title('Fit Residuals')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'nlsq_fit.png', dpi=300, bbox_inches='tight')
plt.show()

logger.info(f"Saved NLSQ fit plot to {output_dir / 'nlsq_fit.png'}")

## 6. Bayesian Inference with NUTS

Use the NLSQ solution as a warm-start for Bayesian inference to quantify parameter uncertainty.

In [ ]:
# Set priors (weakly informative around NLSQ solution)
nlsq_params = model_fit.parameters.to_dict()

model_fit.parameters.set_prior('G', 'normal', loc=nlsq_params['G'], scale=200.0)
model_fit.parameters.set_prior('eta_0', 'normal', loc=nlsq_params['eta_0'], scale=100.0)
model_fit.parameters.set_prior('tau_y', 'normal', loc=nlsq_params['tau_y'], scale=10.0)
model_fit.parameters.set_prior('tau_th', 'normal', loc=nlsq_params['tau_th'], scale=5.0)
model_fit.parameters.set_prior('b', 'normal', loc=nlsq_params['b'], scale=0.05)
model_fit.parameters.set_prior('n', 'normal', loc=nlsq_params['n'], scale=0.2)

print("Priors set (centered on NLSQ solution):")
for param_name in ['G', 'eta_0', 'tau_y', 'tau_th', 'b', 'n']:
    prior = model_fit.parameters.get_prior(param_name)
    print(f"{param_name:8s}: {prior['type']} (loc={prior['loc']:.3f}, scale={prior['scale']:.3f})")

In [ ]:
# Run NUTS sampling
logger.info("Starting Bayesian inference (NUTS)...")

# Use moderate sampling for demonstration (increase for production)
bayesian_result = model_fit.fit_bayesian(
    rheo_data,
    test_mode='creep',
    sigma_applied=sigma_fit,
    num_warmup=1000,
    num_samples=2000,
    num_chains=4,
    seed=42
)

logger.info("Bayesian inference complete")

In [ ]:
# Extract posterior summary
from rheojax.utils.bayesian import compute_rhat, compute_ess

posterior = bayesian_result.posterior_samples

print("\n" + "="*60)
print("Bayesian Posterior Summary")
print("="*60)

for param_name in ['G', 'eta_0', 'tau_y', 'tau_th', 'b', 'n']:
    samples = posterior[param_name]
    mean = jnp.mean(samples)
    std = jnp.std(samples)
    q025 = jnp.percentile(samples, 2.5)
    q975 = jnp.percentile(samples, 97.5)
    rhat = compute_rhat(samples.reshape(4, -1))  # num_chains=4
    ess = compute_ess(samples.reshape(4, -1))
    
    true_val = model.parameters.get_value(param_name)
    
    print(f"\n{param_name}:")
    print(f"  Mean ± Std:     {mean:.4f} ± {std:.4f}")
    print(f"  95% CI:         [{q025:.4f}, {q975:.4f}]")
    print(f"  True value:     {true_val:.4f}")
    print(f"  R-hat:          {rhat:.4f}")
    print(f"  ESS:            {ess:.0f}")

## 7. ArviZ Diagnostics

Use ArviZ for comprehensive MCMC diagnostics.

In [ ]:
try:
    import arviz as az
    ARVIZ_AVAILABLE = True
except ImportError:
    logger.warning("ArviZ not installed. Install with: pip install arviz")
    ARVIZ_AVAILABLE = False

if ARVIZ_AVAILABLE:
    # Convert to ArviZ InferenceData
    idata = az.from_dict(
        posterior={k: v.reshape(4, -1) for k, v in posterior.items()}
    )
    
    # Summary statistics
    print("\nArviZ Summary:")
    print(az.summary(idata, hdi_prob=0.95))

In [ ]:
if ARVIZ_AVAILABLE:
    # Trace plot
    az.plot_trace(idata, compact=True)
    plt.tight_layout()
    plt.savefig(output_dir / 'trace_plot.png', dpi=300, bbox_inches='tight')
    plt.show()
    logger.info(f"Saved trace plot to {output_dir / 'trace_plot.png'}")

In [ ]:
if ARVIZ_AVAILABLE:
    # Pair plot (correlations)
    az.plot_pair(
        idata,
        var_names=['G', 'eta_0', 'tau_y', 'tau_th', 'b', 'n'],
        kind='hexbin',
        marginals=True,
        figsize=(12, 12)
    )
    plt.tight_layout()
    plt.savefig(output_dir / 'pair_plot.png', dpi=300, bbox_inches='tight')
    plt.show()
    logger.info(f"Saved pair plot to {output_dir / 'pair_plot.png'}")

In [ ]:
if ARVIZ_AVAILABLE:
    # Forest plot (credible intervals)
    az.plot_forest(
        idata,
        var_names=['G', 'eta_0', 'tau_y', 'tau_th', 'b', 'n'],
        combined=True,
        hdi_prob=0.95,
        figsize=(8, 6)
    )
    plt.tight_layout()
    plt.savefig(output_dir / 'forest_plot.png', dpi=300, bbox_inches='tight')
    plt.show()
    logger.info(f"Saved forest plot to {output_dir / 'forest_plot.png'}")

## 8. Elastic vs Viscous Strain Analysis

Decompose the creep response to understand Maxwell contribution.

In [ ]:
# Use posterior mean for predictions
G_post = jnp.mean(posterior['G'])
eta_0_post = jnp.mean(posterior['eta_0'])
tau_y_post = jnp.mean(posterior['tau_y'])

# Simulate with posterior mean parameters
model_post = FluiditySaramitoLocal(coupling="minimal")
model_post.parameters.set_values(
    G=float(G_post),
    eta_0=float(eta_0_post),
    tau_y=float(tau_y_post),
    tau_th=float(jnp.mean(posterior['tau_th'])),
    b=float(jnp.mean(posterior['b'])),
    n=float(jnp.mean(posterior['n']))
)

gamma_post, f_post = model_post.simulate_creep(t, sigma_fit)

# Decompose strain
gamma_elastic_post = sigma_fit / G_post
gamma_viscous_post = gamma_post - gamma_elastic_post

# Compute effective viscosity
eta_eff = eta_0_post / f_post

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Panel 1: Strain decomposition
ax = axes[0, 0]
ax.plot(t, gamma_post, 'k-', label='Total', linewidth=2)
ax.axhline(gamma_elastic_post, color='green', linestyle='--', 
           label=f'Elastic (σ/G = {gamma_elastic_post:.4f})', linewidth=2)
ax.plot(t, gamma_viscous_post, 'purple', linestyle=':', 
        label='Viscous', linewidth=2)
ax.plot(fit_data['t'], fit_data['gamma_noisy'], 'o', color='gray', 
        markersize=3, alpha=0.3, label='Data')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Strain γ (-)')
ax.set_title('Strain Decomposition (Posterior Mean)')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 2: Fluidity evolution
ax = axes[0, 1]
ax.plot(t, f_post, 'b-', linewidth=2)
ax.axhline(1.0, color='gray', linestyle='--', alpha=0.5, label='Equilibrium (f=1)')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Fluidity f (-)')
ax.set_title('Fluidity Evolution')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 3: Effective viscosity
ax = axes[1, 0]
ax.plot(t, eta_eff, 'r-', linewidth=2)
ax.axhline(eta_0_post, color='gray', linestyle='--', alpha=0.5, 
           label=f'η₀ = {eta_0_post:.1f} Pa·s')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Effective Viscosity η_eff (Pa·s)')
ax.set_title('Viscosity Evolution (η_eff = η₀/f)')
ax.legend()
ax.grid(True, alpha=0.3)

# Panel 4: Strain rate
ax = axes[1, 1]
gamma_dot_viscous = jnp.gradient(gamma_viscous_post, t)
ax.plot(t, gamma_dot_viscous, 'orange', linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Viscous Strain Rate (1/s)')
ax.set_title('Viscous Flow Rate')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'strain_decomposition.png', dpi=300, bbox_inches='tight')
plt.show()

logger.info(f"Saved strain decomposition to {output_dir / 'strain_decomposition.png'}")

## 9. Save Results

In [ ]:
# Save synthetic data
for label, data in creep_data.items():
    np.savetxt(
        output_dir / f'synthetic_creep_{label}.csv',
        np.column_stack([data['t'], data['gamma'], data['fluidity']]),
        header='time,strain,fluidity',
        delimiter=',',
        comments=''
    )

# Save NLSQ results
with open(output_dir / 'nlsq_results.txt', 'w') as f:
    f.write("NLSQ Fit Results\n")
    f.write("=" * 60 + "\n")
    f.write(f"Converged: {result.success}\n")
    f.write(f"Iterations: {result.nit}\n")
    f.write(f"R²: {result.r_squared:.6f}\n")
    f.write(f"RMSE: {result.rmse:.6e}\n")
    f.write("\nFitted Parameters:\n")
    for param_name in ['G', 'eta_0', 'tau_y', 'tau_th', 'b', 'n']:
        val = model_fit.parameters.get_value(param_name)
        f.write(f"{param_name} = {val:.6f}\n")

# Save Bayesian results
if ARVIZ_AVAILABLE:
    az.to_netcdf(idata, output_dir / 'bayesian_results.nc')
    logger.info(f"Saved ArviZ InferenceData to {output_dir / 'bayesian_results.nc'}")

# Save posterior samples
posterior_df = {k: np.array(v) for k, v in posterior.items()}
import pandas as pd
pd.DataFrame(posterior_df).to_csv(output_dir / 'posterior_samples.csv', index=False)

logger.info(f"All results saved to {output_dir}")

## 10. Key Takeaways

### Maxwell Creep Behavior

1. **Elastic Jump (Instantaneous Response):**
   - All stress levels exhibit γ_e(0) = σ₀/G at t=0
   - This is a signature of the Maxwell viscoelastic backbone
   - Measurable in experiments as the initial instrument compliance

2. **Viscous Bifurcation (Yield Threshold):**
   - **Below yield (σ < τ_y):** Only elastic strain, no flow
   - **At yield (σ ≈ τ_y):** Transient competition between aging and rejuvenation
   - **Above yield (σ > τ_y):** Delayed yielding → terminal flow

3. **Thixotropic Dynamics:**
   - Fluidity increases above yield (rejuvenation dominates)
   - Effective viscosity η_eff = η₀/f decreases over time
   - Flow accelerates: γ̇_v(t) increases even at constant σ₀

4. **Parameter Identifiability:**
   - **G:** Elastic jump magnitude
   - **τ_y:** Bifurcation threshold
   - **η₀, τ_th, b, n:** Terminal flow rate and transient dynamics

### Practical Implications

- **Creep testing** is ideal for measuring yield stress and thixotropic timescales
- **Multi-stress protocol** (below/at/above yield) provides comprehensive characterization
- **Elastic jump** can be used to independently measure G from creep data
- **Bayesian inference** quantifies uncertainty in yield stress determination

### Comparison with Other Protocols

| Protocol | Best For | Elastic Jump? |
|----------|----------|---------------|
| **Creep** | Yield stress, thixotropy | Yes (t=0) |
| **Startup** | Stress overshoot, flow instabilities | No (controlled strain) |
| **SAOS** | Linear viscoelasticity, G'/G'' | Embedded in moduli |
| **Flow Curve** | Steady-state rheology, shear-thinning | No (steady state) |

### Next Steps

- Compare creep-derived parameters with startup and flow curve results
- Explore shear banding in nonlocal creep (1D spatial gradients)
- Test predictive power: use creep-calibrated model for LAOS simulations